In [7]:
import os
import sklearn_crfsuite
from sklearn_crfsuite import metrics

file_path = "spacing_data.txt"

with open(file_path, "r", encoding="utf8") as inFile:
    lines = inFile.readlines()
    
# 데이터를 음절로 이루어진 문장과 정답 값으로 나누어 저장
datas = []
for line in lines:
    pieces = line.strip().split("\t")
    eumjeol_sequence, label = pieces[0].split(), pieces[1].split()
    datas.append((eumjeol_sequence, label))

number_of_train_datas = int(len(datas)*0.9)

train_datas = datas[:number_of_train_datas]
test_datas = datas[number_of_train_datas:]

print("train개수: ", len(train_datas))
print("test개수: ", len(test_datas))

train개수:  900
test개수:  100


In [8]:
# 데이터 변환
def sent2feature(eumjeol_sequence):
    features = []
    sequence_length = len(eumjeol_sequence)
    for index, eumjeol in enumerate(eumjeol_sequence):
        feature = {"BOS": False, "EOS": False, "WORD": eumjeol, "IS_DIGIT": eumjeol.isdigit()}
        
        if(index == 0):
            feature["BOS"] = True
        elif(index == sequence_length-1):
            feature["EOS"] = True
        
        if(index-1 <= sequence_length-1):
            feature["-1_WORD"] = eumjeol_sequence[index-1]
        if(index-2 <= sequence_length-1):
            feature["-2_WORD"] = eumjeol_sequence[index-2]
            
        if(index+1 <= sequence_length-1):
            feature["+1_WORD"] = eumjeol_sequence[index+1]
        if(index+2 <= sequence_length-1):
            feature["+2_WORD"] = eumjeol_sequence[index+2]
        
        features.append(feature)
    
    return features

In [9]:
# 데이터 생성
train_x, train_y = [], []
for eumjeol_sequence, label in train_datas:
    train_x.append(sent2feature(eumjeol_sequence))
    train_y.append(label)

test_x, test_y = [], []
for eumjeol_sequence, label in test_datas:
    test_x.append(sent2feature(eumjeol_sequence))
    test_y.append(label)

In [10]:
# CRFs 학습
crf = sklearn_crfsuite.CRF()
crf.fit(train_x, train_y)

CRF()

In [11]:
# CRFs 평가

def show_predict_result(test_datas, predict):
    for index_1 in range(len(test_datas)):
        eumjeol_sequence, correct_labels = test_datas[index_1]
        predict_labels = predict[index_1]
        
        correct_sentence, predict_sentence = "", ""
        for index_2 in range(len(eumjeol_sequence)):
            if(index_2==0):
                correct_sentence += eumjeol_sequence[index_2]
                predict_sentence += eumjeol_sequence[index_2]
                continue
            
            if(correct_labels[index_2]=="B"):
                correct_labels += " "
            correct_sentence += eumjeol_sequence[index_2]
            
            if(predict_labels[index_2] == "B"):
                predict_sentence += " "
            predict_sentence += eumjeol_sequence[index_2]
        
        print("정답 문장: " + correct_sentence)
        print("출력 문장: " + predict_sentence)
        print()
    


In [12]:
predict = crf.predict(test_x)
print("ACC Score"+ str(metrics.flat_accuracy_score(test_y, predict)))
print()
print("10개의 데이터에 대한 모델 출력과 실제 정답 비교")
print()

show_predict_result(test_datas[:10], predict[:10])

ACC Score0.8969858832506676

10개의 데이터에 대한 모델 출력과 실제 정답 비교

정답 문장: 1914-18년의전쟁은인류를통합시킨최초의공통분모였다.
출력 문장: 1914- 18년의 전쟁은 인류를 통합시킨 최초의 공통분 모였다.

정답 문장: 하지만이전쟁은죽음을통해인류를통합시켰다.
출력 문장: 하지만이 전쟁은 죽음을 통해 인류를 통합시켰다.

정답 문장: 사라예보에서한세르비아인이쏜총한발이합스부르크가의계승자를죽였다.
출력 문장: 사라 예보에서 한 세르 비아인이 쏜총한 발이 합스부르크가의 계승 자를 죽였다.

정답 문장: 이암살행위는국지적인민족주의들과세계적인제국주의들이충돌하는분쟁지역에서저질러졌다.
출력 문장: 이암 살행 위는 국지적인 민족주의 들과 세계적인 제국주의 들이 충돌하는 분쟁 지역에서 저질러졌다.

정답 문장: 오토만제국의점진적인해체는민족주의의독기를발산하는동시에오스트리아,헝가리와독일,영국,프랑스의탐욕을자극했다.
출력 문장: 오토만 제국의 점진 적인 해체는 민족주의의 독기를 발산하는 동시에 오스트리아, 헝가리와 독일, 영국, 프랑스의 탐욕을 자극했다.

정답 문장: 이렇게해서발칸반도의한외진장소에서벌어진국지적인테러행위는일련의긴박한반응을불러일으키면서전유럽에영향을미쳤을뿐만아니라이번에는아시아와아프리카식민지들,일본,그리고이어서미국과멕시코까지끌어들였다.
출력 문장: 이렇게 해서 발칸 반도의 한외진 장소에서 벌어 진국지적인 테러행 위는 일련의 긴박한 반응을 불러일으키면서 전유럽에 영향을 미쳤을 뿐만 아니라이 번에는 아시아와 아프리카 식민지들, 일본, 그리고 이어서 미국과 멕시코까지 끌어들였다.

정답 문장: 전쟁의물결이지구상의모든대양으로밀려드는동안캐나다인들과미국인들,오스트레일리아인들,세네갈인들,알제리인들,모로코인들,안남(安南)인들은연합군깃발을휘날리며유럽전선에서싸웠다.
출력 문장: 전쟁의 물결이 지구상의 모든 대양으로 밀려드는 동안 캐나다인들과 미국인들, 오스트레 일리 아인들, 세네 갈인들, 알 제리인들, 모로 코인들, 안 남(安南)인들은 연합군 깃